# Fusão de novos eventos

Esse notebook é usado para servir de base para um script de extração de eventos do Flurry em formatos de dataframe que serão usados para criar features para: usuários e eventos.

A origem do evento é o Flurry, uma plataforma mobile app analytics, que registra os eventos realizados pelas pessoas que estão usando o aplicativo em produção. O primeiro evento registrado foi em Setembro de 2017. Diariamente são extraídos da plataforma em um formato *json*. Os arquivos tem o seguinte formato:

<img src="../imagens/json-format-flurry.png">


Atualmente esses arquivos são exportados para um parquet (um formato de armazenamento colunar) bastante eficiente em espaço. No entanto para o propósito atual, o ideal é que esse notebook se torne um script (ou parte de um) que atualize os arquivos resultados (que serão apresentados a seguir) diariamente (ou em outra periodicidade que faça sentido).

No atual parquet que é gerado atualmente são coletados apenas o nome do evento, o id do usuário, o modelo do dispositivo do usuário e o país onde foi registrado o evento.

Os arquivos que são criados tem como finalidade apoiar os processos de analise de dados e feature engineering.

**USERS**

* nome do arquivo - *users_YYYY-MM-DD.csv* - 
Eventos com dados de todos os usuários que já registraram algum evento pelo Flurry. A observação é o *userId* e as variáveis são: *deviceModel* (o principal modelo de aparelho em que foi registrado o evento do app) e *countryISO* (o principal país onde o usuário usou o app).

<img src="../imagens/dataframe-users.png">

**EVENTS**
* nome do arquivo - *events_YYYY-MM-DD.csv* - 
Informações sobre todos os eventos registrados pelo flurry. A observação é o nome do evento: *eventName*. As seguintes features foram coletadas dos eventos: *firstOccurrence* (primeira data de ocorrência do evento), *lastOccurrence* (última data de ocorrência do evento), *intervalOccurrence* (diferença em números de dias entre as data anteriores), *aaaa-mm* dezenas de colunas anotando a quantidade de eventos registrados nos meses desde o lançamento do app.

<img src="../imagens/dataframe-events.png">

**EVENTS_PER_USER**
* nome do arquivo - *events_per_user_YYYY-MM-DD.csv* - 
Informações resumidas sobre todos os eventos registrados dos usuários. As linhas são a tupla (*userId* e *eventDate*), ou seja, o usuário a data em que ele tem algum evento registrado. As *colunas são todos os eventos*, onde para cada chave (userId, eventData) é totalizado a quantidade de eventos realizados no dia.

<img src="../imagens/dataframe-events-per-user.png">

## Inicialização

Definindo biblotecas e variáveis

Um pre-requisito para esse notebook é a instalação do fastparquet ou outra biblioteca similar que possa ler o arquivo parquet. As demais bibliotecas são inicializadas a seguir.

As varíaveis iniciais informam o nome dos arquivos anteriores e o nome do arquivo parquet do qual serão extraídos os novos eventos. 

In [2]:
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta
from pathlib import Path

In [3]:
# Define variáveis com o nome e localização dos arquivos da última extração

data_path = Path('/home/wesley/Github/ribon-parquet')

csv_eventFile = 'ribon_events_per_user_2020-04-12.csv'
csv_usersFile = 'ribon_users_2020-04-12.csv'

new_parquet_file = 'flurry_user_event_date_2020_04_17.parquet'

## Carregando os dados para realizar fusão

#### Carregando os últimos dados extraídos em formato de dataframe

In [4]:
# Carrega o último dataframe de usuários e exibe informações básicas
users = pd.read_csv(data_path/csv_usersFile)
print(users.shape)
users.info(),
users.head()

(207287, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207287 entries, 0 to 207286
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   userId       207287 non-null  int64 
 1   deviceModel  207287 non-null  object
 2   countryISO   207287 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.7+ MB


,userId,deviceModel,countryISO
0,1,Moto G (5S) Plus,BR
1,2,Galaxy Note8,BR
2,8,Galaxy S10,BR
3,9,Galaxy S8,BR
4,15,Galaxy S8+,BR


In [5]:
# Carrega dataframe de eventos, transforma a coluna eventDate em data e exibe informações básicas do dataframe
events_per_user = pd.read_csv(data_path/csv_eventFile, parse_dates=['eventDate'])
events_per_user.sort_values('eventDate', inplace=True)
print(events_per_user.shape)
events_per_user.info(),
events_per_user.head()

(3744464, 199)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3744464 entries, 0 to 3744463
Columns: 199 entries, userId to PromotionsSawPromoCard
dtypes: datetime64[ns](1), float64(197), int64(1)
memory usage: 5.6 GB


,userId,ativacao,uncaught,eventDate,Opened,donate,onBoardingStage,openCurtain,changeTabPrincipal,associateWithFacebook,...,SubscribeFromCollectedBonusModal,SubscritionClickedEmptyBundle,SubscritionClickedEmptyBundleRedirect,SubscriptionClickedRedirectFromReceivedAndroid,PromotionsClickedActiveIcon,PromotionsClickedCTA,PromotionsClickedInactiveIcon,PromotionsSawEmptyScreen,PromotionsClickedSocial,PromotionsSawPromoCard
0,1111,6.0,2.0,2017-08-28,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,123,NaN,2.0,2017-10-02,10.0,2.0,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,NaN,1.0,2017-10-02,3.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,NaN,3.0,2017-10-02,4.0,NaN,8.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,2017-10-03,4.0,NaN,10.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Carregando os dados mais recentes do parquet

In [6]:
%%time
# carregando o novo arquivo, com os dados mais recentes
df = pd.read_parquet(data_path/new_parquet_file)

CPU times: user 41.8 s, sys: 7.54 s, total: 49.4 s
Wall time: 43.7 s


In [7]:
# transformando a coluna sessionTimestamp para o formato de data
df['sessionTimestamp'] = pd.to_datetime(df.sessionTimestamp, unit='ms')

# exibindo informações básicas do novo dataframe
print(df.shape)
df.info()

(59877344, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59877344 entries, 0 to 59877343
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   sessionTimestamp  datetime64[ns]
 1   deviceModel       object        
 2   countryISO        object        
 3   eventName         object        
 4   eventOffset       object        
 5   eventParameters   object        
 6   userId            object        
dtypes: datetime64[ns](1), object(6)
memory usage: 3.1+ GB


In [8]:
# Definindo a data de início dos eventos do novo dataframe, os quais serão adicionados ao antigo dataframe 

# Identifica a última data do dataframe inicial
lastEventDate = events_per_user.sort_values('eventDate').eventDate.iloc[-1]

# Estabelece a data inicial para o novo dataframe
data_inicio = (lastEventDate + timedelta(days=1)).date()
data_inicio

datetime.date(2020, 4, 13)

In [9]:
# Ignora os eventos anteriores, pois já existem no dataframe original
df = df[df.sessionTimestamp  >= (lastEventDate + timedelta(days=1))]

In [10]:
print(df.shape, df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419702 entries, 59378943 to 59877343
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   sessionTimestamp  419702 non-null  datetime64[ns]
 1   deviceModel       419702 non-null  object        
 2   countryISO        419702 non-null  object        
 3   eventName         419702 non-null  object        
 4   eventOffset       419702 non-null  object        
 5   eventParameters   419702 non-null  object        
 6   userId            419702 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 25.6+ MB
(419702, 7) None


In [11]:
# Identifica a última data dos eventos que serão adicionados ao anterior

# pega a última data do novo dataframe
lastEventDate = df.sort_values('sessionTimestamp').sessionTimestamp.iloc[-1]
data_fim = (lastEventDate).date()
data_fim

datetime.date(2020, 4, 16)

## Limpeza dos dados do dataframe com os novos dados

In [12]:
# reindexa o novo dataframe
df.head()

,sessionTimestamp,deviceModel,countryISO,eventName,eventOffset,eventParameters,userId
59378943,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,306,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760
59378944,2020-04-13 16:15:18.116,Galaxy J8,BR,ProfileClickedReceiveBonus,5002,"{""userId"":""231760"",""streak"":""8""}",231760
59378945,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,27549,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760
59378946,2020-04-13 16:15:18.116,Galaxy J8,BR,ClickedCollectFeed,33279,"{""screen"":""Feed"",""feedId"":""1323"",""userId"":""231...",231760
59378947,2020-04-13 16:15:18.116,Galaxy J8,BR,OpenedDonationModalFromCard,42813,"{""identifier"":""Ação da Cidadania"",""userId"":""23...",231760


In [13]:
# Analisa os dados não válidos
print('Numero de eventos Nulos: ', df.eventName.isnull().sum())
print('Numero de userId Nulos: ', df.userId.isnull().sum())
print('Numero de registros com eventos ou Ids nulos: ',\
      df[df.eventName.isnull() | df.userId.isnull()].shape[0])

Numero de eventos Nulos:  0
Numero de userId Nulos:  0
Numero de registros com eventos ou Ids nulos:  0


In [14]:
# Analisa eventos com userId não numéricos
df.userId.str.contains('\D+').sum()

23

In [15]:
%%time
# Realiza a limpeza dos dados, removendo os eventos com userId não numérico
df = df[df.userId.str.contains('^\d+$')]
df.userId.str.contains('\D+').sum(), df.shape

CPU times: user 524 ms, sys: 0 ns, total: 524 ms
Wall time: 523 ms


(0, (419679, 7))

In [16]:
# transforma o userID em dados numéricos para deixar no mesmo formato do dataframe anterior
df['userId'] = pd.to_numeric(df['userId'], errors='coerce')

In [17]:
# Analisa a quantidade de eventos com userId igual a zero
print('Nro registros com userID = 0 : ', df.loc[df['userId'] == 0].shape[0])

Nro registros com userID = 0 :  0


In [18]:
# Remove os usuários com o userId igual a zero
df = df.loc[df['userId'] != 0]
df.loc[df['userId'] == 0].shape[0], df.shape

(0, (419679, 7))

In [19]:
df.userId.isnull().sum()

0

In [20]:
%%time
# Reindexa o dataframe com os novos dados após a limpeza
df.reset_index(drop=True, inplace=True)

CPU times: user 119 µs, sys: 0 ns, total: 119 µs
Wall time: 235 µs


In [21]:
df.head()

,sessionTimestamp,deviceModel,countryISO,eventName,eventOffset,eventParameters,userId
0,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,306,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760
1,2020-04-13 16:15:18.116,Galaxy J8,BR,ProfileClickedReceiveBonus,5002,"{""userId"":""231760"",""streak"":""8""}",231760
2,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,27549,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760
3,2020-04-13 16:15:18.116,Galaxy J8,BR,ClickedCollectFeed,33279,"{""screen"":""Feed"",""feedId"":""1323"",""userId"":""231...",231760
4,2020-04-13 16:15:18.116,Galaxy J8,BR,OpenedDonationModalFromCard,42813,"{""identifier"":""Ação da Cidadania"",""userId"":""23...",231760


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419679 entries, 0 to 419678
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   sessionTimestamp  419679 non-null  datetime64[ns]
 1   deviceModel       419679 non-null  object        
 2   countryISO        419679 non-null  object        
 3   eventName         419679 non-null  object        
 4   eventOffset       419679 non-null  object        
 5   eventParameters   419679 non-null  object        
 6   userId            419679 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 22.4+ MB


### Preparando os novos dados para concatenar com o antigo

In [23]:
%%time
# criando a coluna de eventDate para realizar o agrupamento
df['eventDate'] = df.sessionTimestamp.dt.strftime("%Y-%m-%d")

CPU times: user 1.73 s, sys: 0 ns, total: 1.73 s
Wall time: 1.73 s


In [24]:
df.head()

,sessionTimestamp,deviceModel,countryISO,eventName,eventOffset,eventParameters,userId,eventDate
0,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,306,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760,2020-04-13
1,2020-04-13 16:15:18.116,Galaxy J8,BR,ProfileClickedReceiveBonus,5002,"{""userId"":""231760"",""streak"":""8""}",231760,2020-04-13
2,2020-04-13 16:15:18.116,Galaxy J8,BR,SendOneSignalUserId,27549,"{""userId"":"""",""oneSignalUserId"":""af99030a-8a35-...",231760,2020-04-13
3,2020-04-13 16:15:18.116,Galaxy J8,BR,ClickedCollectFeed,33279,"{""screen"":""Feed"",""feedId"":""1323"",""userId"":""231...",231760,2020-04-13
4,2020-04-13 16:15:18.116,Galaxy J8,BR,OpenedDonationModalFromCard,42813,"{""identifier"":""Ação da Cidadania"",""userId"":""23...",231760,2020-04-13


In [25]:
# Realizando o agrupamento pela data
e = df[['eventDate','userId','eventName', 'eventOffset']]\
                  .groupby(['eventDate'], as_index=True)

In [26]:
%%time

# Criando o dataframe new_events_per_user com os novos dados
# Ele será concatenado com os eventos anteriores

new_events_per_user = pd.DataFrame()
for date, group in e:
    #print(name)
    df1 = group.groupby(['userId', 'eventName'], as_index=False).agg({'eventOffset': 'count'})
    df1.columns = ['userId', 'eventName', 'eventOccurrences']
    df1 = df1.pivot(index='userId', columns='eventName', values='eventOccurrences')
    df1.reset_index(inplace=True)
    df1['eventDate'] = date
    #print(group.columns)
    new_events_per_user = pd.concat([new_events_per_user, df1], sort=False, ignore_index=True)

CPU times: user 347 ms, sys: 107 ms, total: 453 ms
Wall time: 737 ms


In [27]:
# estrutura do novo dataframe
new_events_per_user.shape

(27444, 106)

In [28]:
# as colunas do novo dataframe
new_events_per_user.columns

Index(['userId', 'ClickedAdAfterDonate', 'ClickedAdBeforeDonate',
       'ClickedAdMoreInfo', 'ClickedButtonComic', 'ClickedCollectBonus',
       'ClickedCollectFeed', 'ClickedCollectedBonus', 'ClickedCollectedFeed',
       'ClickedEarnedBadge',
       ...
       'openNgoProfileFromCard', 'uncaught', 'eventDate',
       'DonationClickedMoreInfo3', 'PromotionsClickedSocial', 'SkipQuiz',
       'goChatFromFAQ', 'requestDeleteAccount', 'EditProfileChangePassword',
       'OpenedAdOnFeedView'],
      dtype='object', length=106)

In [29]:
# Exibinado algumas linhas do novo dataframe
new_events_per_user[['userId', 'eventDate', 'Doou']].head(10)

,userId,eventDate,Doou
0,9,2020-04-13,3.0
1,23,2020-04-13,1.0
2,58,2020-04-13,5.0
3,59,2020-04-13,NaN
4,68,2020-04-13,1.0
5,149,2020-04-13,NaN
6,232,2020-04-13,NaN
7,416,2020-04-13,NaN
8,459,2020-04-13,NaN
9,489,2020-04-13,NaN


In [30]:
new_events_per_user.head()

,userId,ClickedAdAfterDonate,ClickedAdBeforeDonate,ClickedAdMoreInfo,ClickedButtonComic,ClickedCollectBonus,ClickedCollectFeed,ClickedCollectedBonus,ClickedCollectedFeed,ClickedEarnedBadge,...,openNgoProfileFromCard,uncaught,eventDate,DonationClickedMoreInfo3,PromotionsClickedSocial,SkipQuiz,goChatFromFAQ,requestDeleteAccount,EditProfileChangePassword,OpenedAdOnFeedView
0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2020-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,2020-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,2020-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59,NaN,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,NaN,NaN,2020-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,2020-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Quantidade de usuários unicos no novo dataframe
new_events_per_user.userId.nunique()

13840

### Concatenando os novos eventos com os eventos antigos

In [32]:
%%time
total = pd.concat([events_per_user, new_events_per_user], sort=False, ignore_index=True)

CPU times: user 7.82 s, sys: 5.18 s, total: 13 s
Wall time: 23.9 s


In [33]:
# Exibindo dados do dataframe concatenado
print(total.shape,total.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3771908 entries, 0 to 3771907
Columns: 203 entries, userId to ProfileClickedProfile
dtypes: float64(201), int64(1), object(1)
memory usage: 5.7+ GB
(3771908, 203) None


In [34]:
total.head()

,userId,ativacao,uncaught,eventDate,Opened,donate,onBoardingStage,openCurtain,changeTabPrincipal,associateWithFacebook,...,PromotionsClickedActiveIcon,PromotionsClickedCTA,PromotionsClickedInactiveIcon,PromotionsSawEmptyScreen,PromotionsClickedSocial,PromotionsSawPromoCard,DonationSawSubscriptionPackage,ImpactSawCommunityImpact,ImpactSawUserImpact,ProfileClickedProfile
0,1111,6.0,2.0,2017-08-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,123,NaN,2.0,2017-10-02 00:00:00,10.0,2.0,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,NaN,1.0,2017-10-02 00:00:00,3.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,NaN,3.0,2017-10-02 00:00:00,4.0,NaN,8.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,2017-10-03 00:00:00,4.0,NaN,10.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Exibindo a quantidade de usuários único no dataframe concatenado
total.userId.nunique()

208716

In [60]:
# Existiam duas colunas de doação: donate e Doou
# Se estiver consolidando dataframes mais antigos (Março de 2019), deve ser executadas as linhas abaixo

# events_per_user['Doou'] = events_per_user[['donate','Doou']].sum(axis=1)
# events_per_user.drop(['donate'], axis=1, inplace=True)

In [61]:
# Salvando o novo arquivo com os dados concatenados
arq_nome = 'ribon_events_per_user_' +  str(data_fim) + '.csv'
total.to_csv(data_path/arq_nome, index=False)

### Concatenando os novos usuários ao antigo dataframe de users

In [38]:
%%time

# Pegando os usuários e as features do dataframe com os novos eventos
new_users = df[['userId','deviceModel', 'countryISO']].groupby(['userId']).agg(pd.Series.mode)
print(new_users.shape)
new_users.head()

(13840, 2)
CPU times: user 3.58 s, sys: 35.7 ms, total: 3.61 s
Wall time: 3.75 s


,deviceModel,countryISO
userId,,
1,Moto G (5S) Plus,BR
8,Galaxy S10,BR
9,Galaxy S8,BR
23,iPhone XR,BR
25,Galaxy J7 Prime,BR


In [39]:
# Definindo o userId como index do dataframe antigo para preparar a concatenação
users.set_index('userId', inplace=True)

In [40]:
# Informação do novo dataframe
new_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13840 entries, 1 to 268364
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   deviceModel  13840 non-null  object
 1   countryISO   13840 non-null  object
dtypes: object(2)
memory usage: 324.4+ KB


In [41]:
# Informação do antigo dataframe
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207287 entries, 1 to 99996
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   deviceModel  207287 non-null  object
 1   countryISO   207287 non-null  object
dtypes: object(2)
memory usage: 4.7+ MB


In [42]:
users.head()

,deviceModel,countryISO
userId,,
1,Moto G (5S) Plus,BR
2,Galaxy Note8,BR
8,Galaxy S10,BR
9,Galaxy S8,BR
15,Galaxy S8+,BR


In [43]:
new_users.head()

,deviceModel,countryISO
userId,,
1,Moto G (5S) Plus,BR
8,Galaxy S10,BR
9,Galaxy S8,BR
23,iPhone XR,BR
25,Galaxy J7 Prime,BR


In [44]:
# concatenando o novo com o antigo
# a ordem usava foi essa para manter os dados do novo dataframe em relação ao antigo
total_users = pd.concat([new_users, users], axis=0).reset_index().drop_duplicates(subset='userId', keep='first')
print(total_users.shape, total_users.info())
total_users.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208716 entries, 0 to 221126
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   userId       208716 non-null  int64 
 1   deviceModel  208716 non-null  object
 2   countryISO   208716 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.4+ MB
(208716, 3) None


,userId,deviceModel,countryISO
0,1,Moto G (5S) Plus,BR
1,8,Galaxy S10,BR
2,9,Galaxy S8,BR
3,23,iPhone XR,BR
4,25,Galaxy J7 Prime,BR


In [45]:
# Salvando o novo arquivo de usuários
arq_nome = 'ribon_users_' +  str(data_fim) + '.csv'

total_users.to_csv(data_path/arq_nome, index=False)

In [46]:
data_fim

datetime.date(2020, 4, 16)

## Criando o novo arquivo Events

In [70]:
# definindo duas matrizes uma com as colunas que são de eventos e a outra que não são
NO_EVENTS = ['userId', 'eventDate']
EVENTS = [c for c in events_per_user.columns if c not in NO_EVENTS]
len(EVENTS)

196

In [79]:
# Vamos criar 3 features para cada evento: firstOccurrence', 'lastOccurrence', 'intervalOccurrence'

# iniciando o DataFrame de eventos, com as colunas selecionadas para a analise
events = pd.DataFrame(columns=['eventName', 'firstOccurrence', 'lastOccurrence', 'intervalOccurrence'])

# para cada evento é criado uma linha com o calculo das datas e do intervalo
for e in EVENTS:
    
    firstOccurrence = events_per_user.eventDate[events_per_user[e] > 0].sort_values().iloc[0]
    lastOccurrence = events_per_user.eventDate[events_per_user[e] > 0].sort_values().iloc[-1]
    
    events = events.append({'eventName': e,
                              'firstOccurrence': firstOccurrence,
                              'lastOccurrence': lastOccurrence,
                              'intervalOccurrence': lastOccurrence - firstOccurrence},
                              ignore_index=True)
    
# Converte o intervalo de Timedelta para float
events.intervalOccurrence = events.intervalOccurrence.apply(lambda x: float(x.days))

events.shape, events.head()

((196, 4),
          eventName firstOccurrence lastOccurrence  intervalOccurrence
 0         ativacao      2017-08-28     2017-08-28                 0.0
 1         uncaught      2017-08-28     2020-04-12               958.0
 2           Opened      2017-10-02     2020-04-10               921.0
 3  onBoardingStage      2017-10-02     2019-05-27               602.0
 4      openCurtain      2017-10-02     2020-01-16               836.0)

In [80]:
# definindo uma matriz para definir as colunas que serão usadas (filtradas) antes do resample
# basicamente adicionando o eventDate com a matriz EVENTS
colunas = EVENTS
colunas.append('eventDate')

# cria um novo dataframe com a somatória mensal de eventos.
# de forma que os eventos ficam nas linhas e os meses na coluna
eventos_mes = events_per_user[colunas].resample('M', on='eventDate').sum().transpose()

# renomeia as colunas do dataframa só com os dados do ano e mês
eventos_mes.columns = [str(m.date())[:7] for m in eventos_mes.columns]

eventos_mes.shape, eventos_mes.columns, eventos_mes.head()

((196, 33),
 Index(['2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01',
        '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07',
        '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01',
        '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07',
        '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
        '2020-02', '2020-03', '2020-04'],
       dtype='object'),
                  2017-08  2017-09  2017-10  2017-11  2017-12  2018-01  \
 ativacao             6.0      0.0      0.0      0.0      0.0      0.0   
 uncaught             2.0      0.0     89.0    220.0    164.0     39.0   
 Opened               0.0      0.0   3812.0   5496.0   7168.0   8718.0   
 onBoardingStage      0.0      0.0    969.0    653.0   2547.0   3124.0   
 openCurtain          0.0      0.0   2262.0   2611.0   3513.0   3444.0   
 
                  2018-02  2018-03  2018-04  2018-05  ...  2019-07  2019-08  \
 ativacao             0.0  

In [81]:
# juntando os dataframes eventos e o eventos_mes
events = events.sort_values(by=['firstOccurrence'], ascending=True).join(eventos_mes, on= 'eventName')
events.shape, events.head()

((196, 37),
       eventName firstOccurrence lastOccurrence  intervalOccurrence  2017-08  \
 0      ativacao      2017-08-28     2017-08-28                 0.0      6.0   
 1      uncaught      2017-08-28     2020-04-12               958.0      2.0   
 4   openCurtain      2017-10-02     2020-01-16               836.0      0.0   
 71         Doou      2017-10-02     2020-04-12               923.0      0.0   
 2        Opened      2017-10-02     2020-04-10               921.0      0.0   
 
     2017-09  2017-10  2017-11  2017-12  2018-01  ...   2019-07   2019-08  \
 0       0.0      0.0      0.0      0.0      0.0  ...       0.0       0.0   
 1       0.0     89.0    220.0    164.0     39.0  ...   17201.0   35093.0   
 4       0.0   2262.0   2611.0   3513.0   3444.0  ...      14.0      13.0   
 71      0.0   2134.0   3452.0   4674.0   6274.0  ...  406257.0  650544.0   
 2       0.0   3812.0   5496.0   7168.0   8718.0  ...      48.0      22.0   
 
      2019-09   2019-10   2019-11   2019-1

In [86]:
# salvando o arquivo de eventos por usuário
arq_name = 'ribon_events_' + str(data_fim) + '.csv'

events.to_csv(data_path/arq_name, index=False)